## **Embedding Generation Using CLIP (Contrastive Language-Image Pre-training)**

In this notebook, we will:
- Load the pre-processed and augmented images.
- Generate embeddings using the CLIP model.
- Save the embeddings and related data for evaluation..

---

## **Table of Contents**
---
1. Import Libraries
2. Set Up Device
3. Load Augmented Image Mapping
4. Load CLIP Model
5. Generate Embeddings
6. Select Representative Embeddings
7. Save Embeddings
8. Clear Memory
9. Conclusion

---
### **Step 1: Import Libraries**

We begin by importing the necessary libraries.

In [2]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch
from torchvision import transforms
import gc
from transformers import CLIPModel, CLIPProcessor

---
### **Step 2: Set Up Device**

In [3]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


---
### **Step 3: Load Augmented Image Mapping**

In [4]:
# Load augmented image mapping
augmented_df = pd.read_csv('augmented_image_mapping.csv')

---
### **Step 4: Load CLIP Model**

In [5]:
# Load the pre-trained CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_name).to(device)
processor = CLIPProcessor.from_pretrained(model_name)

# Set model to evaluation mode
model.eval()

/Users/mohammedkhodorfirasal-tal/Documents/Professional/Work/Fellowship - Novartis/Data/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e

---
### **Step 5: Generate Embeddings**

In [7]:
# Directory containing augmented images
augmented_dir = './augmented_dataset/'

# Prepare lists to store embeddings and image information
embeddings = []
augmented_image_files = []
original_image_files = []

# Generate embeddings
for idx, row in tqdm(augmented_df.iterrows(), total=len(augmented_df), desc='Generating Embeddings'):
    augmented_image_file = row['augmented_image']
    original_image_file = row['original_image']
    image_path = os.path.join(augmented_dir, augmented_image_file)

    # Load and preprocess image
    img = Image.open(image_path).convert('RGB')
    inputs = processor(images=img, return_tensors='pt').to(device)

    # Generate embedding
    with torch.no_grad():
        outputs = model.get_image_features(**inputs)
        embedding = outputs.cpu().numpy()[0]
        # Normalize embedding
        embedding = embedding / np.linalg.norm(embedding)
        embeddings.append(embedding)
        augmented_image_files.append(augmented_image_file)
        original_image_files.append(original_image_file)

    # Clear memory
    del img, inputs, outputs
    torch.cuda.empty_cache()
    gc.collect()

# Convert embeddings to NumPy array
embeddings = np.vstack(embeddings)

Generating Embeddings: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30240/30240 [29:15:28<00:00,  3.48s/it]


---
### **Step 6: Select Representative Embeddings**

We will use the Highest Norm Criterion to select the most representative embedding for each original image.

In [8]:
# Create a DataFrame for grouping
data = pd.DataFrame({
    'original_image': original_image_files,
    'augmented_image': augmented_image_files,
    'embedding_index': range(len(embeddings))
})

selected_embeddings = []
selected_image_files = []

# Group by original image and select the embedding with the highest norm
for original_image, group in data.groupby('original_image'):
    indices = group['embedding_index'].tolist()
    group_embeddings = embeddings[indices]
    # Compute norms of embeddings
    norms = np.linalg.norm(group_embeddings, axis=1)
    # Select the embedding with the highest norm
    best_idx_in_group = np.argmax(norms)
    best_idx = indices[best_idx_in_group]
    selected_embeddings.append(embeddings[best_idx])
    selected_image_files.append(original_image)

---
### **Step 7: Save Embeddings**

In [9]:
# Convert selected embeddings to NumPy array
selected_embeddings = np.vstack(selected_embeddings)

# Save embeddings and image files
np.save('embeddings_clip.npy', selected_embeddings)
np.save('image_files_clip.npy', selected_image_files)

print('Embeddings for CLIP saved.')

Embeddings for CLIP saved.


---
### **Step 8: Clear Memory**

In [10]:
# Clear variables and free memory
del embeddings, augmented_image_files, original_image_files, data
del selected_embeddings, selected_image_files, model
torch.cuda.empty_cache()
gc.collect()

0

---
### **Conclusion**

We have generated embeddings using the CLIP model, selected representative embeddings, and saved them for evaluation.